# Workflow for Build Workout Dashboard

In [1]:
from build_workout_dashboard.utilities import *
import toml
import os

# Utilities load the pyproject.toml file
with open("pyproject.toml", "r") as f:
    config = toml.load("pyproject.toml")

dbConfig = config['tool']['project']['database']

input_filepath = config['tool']['poetry']['name'].replace('-', '_') + os.path.sep + config['tool']['project']['input_filename']


In [2]:
## Toggle if development to use local host instead of RDS on AWS:
if not(config['tool']['project']['debug']):
    print("Using production database configuration.")
    dbConfig['host'] = os.getenv('RDS_ENDPOINT')
    dbConfig['username'] = os.getenv('RDS_USER')
    dbConfig['password'] = os.getenv('RDS_PASSWORD')


print("Connecting to MySQL database...")
print(dbConfig)
connection = mysql.connector.connect(**dbConfig)
cursor = connection.cursor()

cursor.execute("SHOW DATABASES")
databases = cursor.fetchall()                  # Fetch all results
databases = [item[0] for item in databases]    # Convert tuple responses to a list

print("Databases visible to your user:")
for db in databases:            # Print the databases
    print(db)

Connecting to MySQL database...
{'host': 'localhost', 'port': 8889, 'username': 'barbs', 'password': 'barbs'}
Databases visible to your user:
information_schema
TESTDB
amazon_product_reviews
mysql
performance_schema
running
sweat
sys


In [3]:
dbConfig['database'] = "sweat"

if dbConfig['database'] not in databases:
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {dbConfig['database']};")    
    print(f"Created database: {dbConfig['database']};")  


In [4]:
# Set-up database and main table
setup_database(cursor, dbConfig)


Using this database configuration:
{'host': 'localhost', 'port': 8889, 'username': 'barbs', 'password': 'barbs', 'database': 'sweat'}

Using database: sweat
sweat.workout_summary table has 0 rows.


In [6]:
# Get list of existing workout_ids in Db
cursor = connection.cursor()
cursor.execute(f"USE {dbConfig['database']}")
cursor.execute("SELECT workout_id FROM workout_summary")
existing_workout_ids = [row[0] for row in cursor.fetchall()]
print(f"# rows in workout_summary table: {len(existing_workout_ids)}")


# rows in workout_summary table: 0


# Main function

## Step 1 - Ingest the data file

***"build_workout_dashboard/user2632022_workout_history.csv"***


In [9]:
### Read CSV file
print(f'Reading data from {input_filepath}')
df = pd.read_csv(input_filepath)

### Clean data (drop unnecessary columns)
df = clean_data(df)

# Enrich data (for now, just extract workoutID_
df = enrich_data(df)

Reading data from build_workout_dashboard/user2632022_workout_history_OLD.csv
Dropped 3 rows with zero workout time.
Dropped 144 rows with invalid dates.
Final number of rows: 1952


## Identify new workout_ids that are not in database

In [10]:
# Assuming 'workout_id' is the column in the DataFrame containing workout IDs
newDf = df[~df['workout_id'].isin(existing_workout_ids)]
print(newDf.shape[0])
newDf

1952


,workout_date,activity_type,kcal_burned,distance_mi,duration_sec,avg_pace,max_pace,steps,link,workout_id
0,2023-08-02,Run,686.0,6.25519,3825.0,10.19150,7.60008,10362.0,http://www.mapmyfitness.com/workout/7434147697,7434147697
1,2023-08-01,Run,100.0,1.26712,1664.0,21.88690,10.02760,4226.0,http://www.mapmyfitness.com/workout/7431852028,7431852028
2,2023-07-31,Run,163.0,2.06009,2789.0,22.56380,8.07587,7070.0,http://www.mapmyfitness.com/workout/7429498717,7429498717
3,2023-07-30,Run,98.0,1.05951,1896.0,29.82510,12.98170,4740.0,http://www.mapmyfitness.com/workout/7427597173,7427597173
4,2023-07-29,Run,496.0,4.47639,2808.0,10.45490,7.50191,7585.0,http://www.mapmyfitness.com/workout/7426008661,7426008661
...,...,...,...,...,...,...,...,...,...,...
1947,2013-06-19,Run,384.0,3.33000,1680.0,8.41000,0.00000,0.0,http://www.mapmyfitness.com/workout/309076653,309076653
1948,2013-06-17,Run,0.0,4.96712,3380.0,11.34120,0.00000,0.0,http://www.mapmyfitness.com/workout/305280701,305280701
1949,2011-11-09,Run,0.0,6.80438,2700.0,6.61339,0.00000,0.0,http://www.mapmyfitness.com/workout/76822852,76822852
1950,2011-08-14,Run,1381.0,13.02560,5687.0,7.27682,4.16161,NaN,http://www.mapmyfitness.com/workout/372819401,372819401


## Update the database 

In [11]:
try:
    # Establish MySQL connection
    if 'connection' not in locals():
        print("Connecting to MySQL database...")
        print(dbConfig)
        connection = mysql.connector.connect(**dbConfig)

    # If connection was lost, either reconnect OR re-establish cursor
    if not(connection.is_connected()):
        cursor = connection.cursor()        

    # Insert data
    rows_affected = insert_data_NEW(cursor, newDf)

    # Commit changes
    connection.commit()

    print(f"Data import completed. {rows_affected} rows were inserted.")

    # Get the last inserted id
    cursor.execute("SELECT LAST_INSERT_ID()")
    last_id = cursor.fetchone()[0]
    print(f"The last inserted ID was: {last_id}")

    # Get the total number of rows in the table
    cursor.execute("SELECT COUNT(*) FROM workout_summary")
    total_rows = cursor.fetchone()[0]
    print(f"Total rows in the table after insert: {total_rows}")

except Error as e:
    print(f"Error: {e}")
    print(f"Error Code: {e.errno}")
    print(f"SQLSTATE: {e.sqlstate}")
    print(f"Message: {e.msg}")
    
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")
        

Data import completed. 1952 rows were inserted.
The last inserted ID was: 0
Total rows in the table after insert: 1952
MySQL connection is closed


## Use connection

In [12]:
# Establish MySQL connection
if not(connection.is_connected()):
    connection = mysql.connector.connect(**dbConfig)
    cursor = connection.cursor()

cursor.execute("DESCRIBE workout_summary;")
cursor.fetchall()

[('workout_id', 'varchar(20)', 'NO', 'PRI', None, ''),
 ('workout_date', 'datetime', 'YES', '', None, ''),
 ('activity_type', 'varchar(50)', 'YES', '', None, ''),
 ('kcal_burned', 'bigint(20)', 'YES', '', None, ''),
 ('distance_mi', 'float', 'YES', '', None, ''),
 ('duration_sec', 'float', 'YES', '', None, ''),
 ('avg_pace', 'float', 'YES', '', None, ''),
 ('max_pace', 'float', 'YES', '', None, ''),
 ('steps', 'bigint(20)', 'YES', '', None, ''),
 ('link', 'varchar(100)', 'YES', '', None, '')]

In [ ]:
newDf